In [7]:
!pip install openai

In [2]:
!pip install chromadb

### Step to extract the data from the jsonl file and insert it to chromadb

In [ ]:
import chromadb
import json

# Initialize ChromaDB client with the default settings
client = chromadb.Client()

# Create or get a collection
collection = client.create_collection("product_management_questions")

# Path to your JSONL file
input_file = 'cleaned_output_file.jsonl'

# Batch size for inserting documents
batch_size = 1000  # Adjust this size as needed

# Function to process and insert a batch of documents
def process_batch(start_idx, lines):
    documents = []
    ids = []
    for idx, line in enumerate(lines):
        entry = json.loads(line)
        text = entry['text'].strip()  # Extract the text content

        if text:  # Check if text is not empty
            documents.append(text)
            ids.append(str(start_idx + idx))

    # Insert documents into ChromaDB
    if documents:
        collection.add(documents=documents, ids=ids)

# Main loop to read the file and process in batches
total_inserted = 0
with open(input_file, 'r', encoding='utf-8') as file:
    batch_lines = []
    for idx, line in enumerate(file):
        batch_lines.append(line)

        # When batch size is reached, process the batch
        if len(batch_lines) >= batch_size:
            process_batch(total_inserted, batch_lines)
            total_inserted += len(batch_lines)
            batch_lines = []  # Clear the batch list for the next batch

        # Print progress every 10,000 lines
        if (idx + 1) % 10000 == 0:
            print(f"Processed {idx + 1} lines, inserted {total_inserted} documents so far.")

    # Process any remaining lines that didn't make a full batch
    if batch_lines:
        process_batch(total_inserted, batch_lines)
        total_inserted += len(batch_lines)

print(f"Inserted a total of {total_inserted} documents into ChromaDB.")

# Retrieve some data from the collection to verify
results = collection.get()
for result in results['documents'][:10]:  # Print first 10 stored documents
    print(f"Stored document: {result}")


In [3]:
file_path = 'cleaned_output_file.jsonl'

with open(file_path, 'r', encoding='utf-8') as file:
    line_count = sum(1 for line in file)

print(f"Total number of lines: {line_count}")


Total number of lines: 10555


In [1]:
import openai

In [2]:
import os

In [3]:
os.environ['OPENAI_API_KEY'] = 'your-openai-api-key'

In [11]:
pip install openai speechrecognition pyttsx3


Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pyaudio


Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install chromadb openai requests


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install requests pyaudio openai


Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install speechrecognition pyaudio requests openai


Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install requests pyaudio


Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install chromadb openai requests speechrecognition pyttsx3


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install chromadb openai requests speechrecognition pyttsx3


In [17]:
pip install openai chromadb elevenlabs sounddevice


     -------------------------------------- 129.2/129.2 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 189.8/189.8 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


#### First option of the bot

In [13]:
import chromadb
import random
import openai
import speech_recognition as sr
import pyttsx3
from openai import OpenAI

# Initialize ChromaDB client with the default settings
client = chromadb.Client()

# Access the existing collection
collection = client.get_collection("product_management_questions")

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Function to speak a given text
def speak_text(text):
    engine.say(text)
    engine.runAndWait()

# Function to listen and recognize speech
def listen_to_user():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)

        try:
            return recognizer.recognize_google(audio)
        except sr.UnknownValueError:
            return "Sorry, I did not catch that."
        except sr.RequestError:
            return "Sorry, my speech service is down."

# Function to retrieve random documents from ChromaDB
def get_random_documents(collection, num_documents=5):
    results = collection.get()
    documents = results.get('documents', [])
    if not documents:
        print("No documents found in the collection.")
    random_documents = random.sample(documents, min(num_documents, len(documents)))
    return random_documents

# Function to ask GPT-4 to generate a product management interview question
def generate_product_management_question(documents):
    openai.api_key = 'use-your-openai-api'  # Replace with your OpenAI API key

    # Combine the documents into a single prompt
    combined_text = " ".join(documents)
    prompt = f"Based on the following content, generate a product management interview question:\n\n{combined_text}"

    client = OpenAI()

    # Use OpenAI's API to generate the question with GPT-4
    response = client.chat.completions.create(
        model="gpt-4",  # Specify GPT-4 model
        messages=[
            {"role": "system", "content": "You are an expert in product management."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,  # Adjust as needed
        n=1,
        stop=None,
        temperature=0,
    )

    # Extract and return the generated question
    generated_question = response.choices[0].message.content.strip()
    return generated_question

# Function to evaluate the user's answer using GPT-4
def evaluate_answer_with_gpt4(question, user_answer):
    openai.api_key = 'use-your-openai-api'  # Replace with your OpenAI API key

    # Formulate the prompt for evaluation
    prompt = f"""
    You are an expert interviewer. Please evaluate the following response based on the provided rubric:

    Question: {question}

    User's Response: {user_answer}

    Rubric:
    1. **How the candidate breaks down a big problem**:
       - 1-2/10: Jumps into solutions without structured thinking.
       - 7-8/10: Brings context, prioritizes solutions, considers measurement and risks.
       - 9-10/10: Demonstrates exceptional structured thinking with clear prioritization, risk management, and metrics for success.

    2. **The candidate's ability to think creatively**:
       - 1-2/10: Comes up with creative feature ideas that may not work.
       - 7-8/10: Thinks of solutions that expand the aperture of what’s possible.
       - 9-10/10: Presents innovative, scalable solutions with clear impact.

    3. **The candidate's tech insight & knowledge**:
       - 1-2/10: Shows no critical thought about tech strategy.
       - 7-8/10: Demonstrates a solid understanding of tech strategy with practical insights.
       - 9-10/10: Offers unique insights and strategic thinking that reflect deep industry knowledge.

    Provide a score out of 10 for each area with a brief explanation, followed by a final overall score out of 10.
    """

    client = OpenAI()
    # Call the OpenAI API to generate the evaluation
    response = client.chat.completions.create(
        model="gpt-4",  # Specify GPT-4 model
        messages=[
            {"role": "system", "content": "You are an expert product management interviewer."},
            {"role": "user", "content": prompt}
        ]
    )

    # Extract the generated evaluation text
    evaluation = response.choices[0].message.content.strip()
    return evaluation

# Main function to combine the steps
def main():
    print("Welcome to the Product Management Bot!")
    speak_text("Welcome to the Product Management Bot!")
    print("Please select an option:")
    speak_text("Please select an option:")
    print("1. Let the bot ask an interview question and evaluate your answer.")
    speak_text("Option 1, let the bot ask an interview question and evaluate your answer.")
    print("2. Provide your own project management interview question and get guidelines on how to answer it.")
    speak_text("Option 2, provide your own project management interview question and get guidelines on how to answer it.")

    choice = input("Enter 1 or 2: ").strip()

    if choice == "1":
        # Retrieve random documents from ChromaDB
        documents = get_random_documents(collection, num_documents=5)
        print(f"Retrieved Documents: {documents}")

        # Generate a product management interview question using GPT-4
        generated_question = generate_product_management_question(documents)
        print(f"Generated Product Management Interview Question:\n{generated_question}")
        speak_text(generated_question)

        # Ask the user to answer the question
        speak_text("Please answer the question.")
        user_answer = listen_to_user()
        print(f"Your Answer: {user_answer}")

        while True:
            speak_text("Do you want to ask any follow-up questions or add more to your answer?")
            follow_up = listen_to_user().lower()
            if "no" in follow_up or "stop" in follow_up:
                break

            # Here, the bot can respond based on its knowledge or ask another question
            response = generate_product_management_question(documents)  # This simulates a follow-up
            print(f"Follow-up Response: {response}")
            speak_text(response)

            user_follow_up_answer = listen_to_user()
            user_answer += " " + user_follow_up_answer
            print(f"Updated Answer: {user_answer}")

        # Evaluate the user's response with GPT-4
        evaluation = evaluate_answer_with_gpt4(generated_question, user_answer)
        print(f"Evaluation Result:\n{evaluation}")
        speak_text(f"Your response was evaluated. Here is the result. {evaluation}")

    elif choice == "2":
        speak_text("Please provide your product management interview question.")
        user_question = listen_to_user()
        print(f"Your Question: {user_question}")

        # Generate guidelines for answering the question with GPT-4
        guidelines = generate_guidelines_with_gpt4(user_question)
        print(f"Guidelines for Answering the Question:\n{guidelines}")
        speak_text(guidelines)

    else:
        print("Invalid option. Please restart the program and choose either 1 or 2.")
        speak_text("Invalid option. Please restart the program and choose either 1 or 2.")

if __name__ == "__main__":
    main()


Welcome to the Project Management Bot!
Please select an option:
1. Let the bot ask an interview question and evaluate your answer.
2. Provide your own project management interview question and get guidelines on how to answer it.
Enter 1 or 2: 1
Retrieved Documents: ['Still, we’ll want to think about the different types of users.', 'While there are ways to spin this in a positive way, being bored at your current role means your most recent experience isn’t very valuable or interesting.', 'You can say “That’s not my job.” While most roles on the team are crisply defined, product managers have a more fluid role.', 'It’s valuable to spend the time to aggregate this feedback to understand trends amongst your user base and what areas might be worth investing in.', 'if (maxSum < runningSum) { 12 .']
Generated Product Management Interview Question:
How would you approach aggregating and analyzing user feedback to understand trends and determine areas worth investing in, especially in a role as

In [24]:
!python --version

Python 3.9.13


In [22]:
!python -m ipykernel install --user --name python312 --display-name "Python 3.12"


Installed kernelspec python312 in C:\Users\poulo\AppData\Roaming\jupyter\kernels\python312


In [23]:
!python --version

Python 3.9.13


In [25]:
!python -m ipykernel install --user --name py311 --display-name "Python 3.11.5 (py311)"


Installed kernelspec py311 in C:\Users\poulo\AppData\Roaming\jupyter\kernels\py311


In [1]:
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [12]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install openai chromadb elevenlabs sounddevice


Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install openai sounddevice numpy


Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install wave

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wave: filename=Wave-0.0.2-py3-none-any.whl size=1220 sha256=e1553621049da315b0938e1125d5c520f3e03dbc1def54e5028199dd2bf92dbb
  Stored in directory: c:\users\poulo\appdata\local\pip\cache\wheels\ee\3e\b3\bdee8d885ec04fd42bd239e9d9ab9dc94e69aad318c0d9936c
Successfully built wave
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install deepgram-sdk speechrecognition chromadb openai python-dotenv aiofiles pydub


Note: you may need to restart the kernel to use updated packages.


#### 2nd option that the users will have

In [10]:
import chromadb
import random
import openai
import speech_recognition as sr
import pyttsx3
import time
from openai import OpenAI

# Initialize ChromaDB client with the default settings
client = chromadb.Client()

# Access the existing collection
collection = client.get_collection("product_management_questions")

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Function to speak a given text
def speak_text(text):
    engine.say(text)
    engine.runAndWait()

# Function to listen and recognize speech
def listen_to_user():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)

        try:
            return recognizer.recognize_google(audio).lower()
        except sr.UnknownValueError:
            return "Sorry, I did not catch that."
        except sr.RequestError:
            return "Sorry, my speech service is down."

# Function to retrieve random documents from ChromaDB
def get_random_documents(collection, num_documents=5):
    results = collection.get()
    documents = results.get('documents', [])
    if not documents:
        print("No documents found in the collection.")
    random_documents = random.sample(documents, min(num_documents, len(documents)))
    return random_documents

# Function to generate a product management interview question
def generate_product_management_question(documents):
    openai.api_key = 'your-openai-api-key'  # Replace with your OpenAI API key

    combined_text = " ".join(documents)
    prompt = f"Based on the following content, generate a product management interview question:\n\n{combined_text}"

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert in product management."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0,
    )
    return response.choices[0].message.content.strip()

# Function to rephrase or elaborate on a product management question
def elaborate_or_rephrase_question(question, action):
    openai.api_key = 'your-openai-api-key'  # Replace with your OpenAI API key

    prompt = f"Please {action} the following product management interview question:\n\n{question}"

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert in product management."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0,
    )
    return response.choices[0].message.content.strip()

# Function to evaluate the user's answer using GPT-4
def evaluate_answer_with_gpt4(question, user_answer):
    openai.api_key = 'your-openai-api-key'  # Replace with your OpenAI API key

    prompt = f"""
    You are an expert interviewer. Please evaluate the following response based on the provided rubric:

    Question: {question}

    User's Response: {user_answer}

    Rubric:
    1. **How the candidate breaks down a big problem**:
       - 1-2/10: Jumps into solutions without structured thinking.
       - 7-8/10: Brings context, prioritizes solutions, considers measurement and risks.
       - 9-10/10: Demonstrates exceptional structured thinking with clear prioritization, risk management, and metrics for success.

    2. **The candidate's ability to think creatively**:
       - 1-2/10: Comes up with creative feature ideas that may not work.
       - 7-8/10: Thinks of solutions that expand the aperture of what’s possible.
       - 9-10/10: Presents innovative, scalable solutions with clear impact.

    3. **The candidate's tech insight & knowledge**:
       - 1-2/10: Shows no critical thought about tech strategy.
       - 7-8/10: Demonstrates a solid understanding of tech strategy with practical insights.
       - 9-10/10: Offers unique insights and strategic thinking that reflect deep industry knowledge.

    Provide a score out of 10 for each area with a brief explanation, followed by a final overall score out of 10.
    """

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert product management interviewer."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# Function to generate guidelines on how to answer a user's question
def generate_guidelines_with_gpt4(user_question):
    openai.api_key = 'your-openai-api-key'  # Replace with your OpenAI API key

    prompt = f"""
    The following is a product management interview question: "{user_question}".
    Please provide a breakdown of how to answer this question with a Good, Better, and Best approach.

    - **Good Answer**: What would a reasonable, competent answer look like?
    - **Better Answer**: How can the answer be improved by showing deeper thought and structure?
    - **Best Answer**: What would an exceptional answer include, demonstrating creativity, deep insight, and strategic thinking?
    """

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert product management interviewer."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,  # Adjust max tokens based on the desired length
        n=1,
        stop=None,
        temperature=0.7,
    )

    return response.choices[0].message.content.strip()

# Helper function to check if input contains certain keywords
def detect_command(user_input):
    if any(keyword in user_input for keyword in ["rephrase", "reword", "restate"]):
        return "rephrase"
    elif any(keyword in user_input for keyword in ["more details", "elaborate", "explain"]):
        return "elaborate"
    elif any(keyword in user_input for keyword in ["follow-up", "follow up", "ask more"]):
        return "follow-up"
    elif any(keyword in user_input for keyword in ["answer", "respond"]):
        return "answer"
    return None

# Main function to combine the steps
def main():
    print("Welcome to the Product Management Bot!")
    speak_text("Welcome to the Product Management Bot!")
    print("Please select an option:")
    speak_text("Please select an option:")
    print("1. Let the bot ask an interview question and evaluate your answer.")
    speak_text("Option 1, let the bot ask an interview question and evaluate your answer.")
    print("2. Provide your own product management interview question and get guidelines on how to answer it.")
    speak_text("Option 2, provide your own product management interview question and get guidelines on how to answer it.")

    choice = input("Enter 1 or 2: ").strip()

    if choice == "1":
        documents = get_random_documents(collection, num_documents=5)
        generated_question = generate_product_management_question(documents)
        print(f"Generated Product Management Interview Question:\n{generated_question}")
        speak_text(generated_question)

        while True:
            speak_text("Do you want to answer the question, ask a follow-up question, ask for more details, or ask to rephrase the question?")
            user_input = listen_to_user()
            command = detect_command(user_input)

            if command == "rephrase":
                generated_question = elaborate_or_rephrase_question(generated_question, "rephrase")
                print(f"Rephrased Question: {generated_question}")
                speak_text(f"Here is the rephrased question: {generated_question}")

            elif command == "elaborate":
                elaborated_question = elaborate_or_rephrase_question(generated_question, "elaborate on")
                print(f"Elaborated Question: {elaborated_question}")
                speak_text(f"Here is more detail: {elaborated_question}")

            elif command == "follow-up":
                speak_text("Please provide your follow-up question.")
                follow_up_question = listen_to_user()
                print(f"Your Follow-up Question: {follow_up_question}")
                # Simulate a response or another question for the follow-up
                follow_up_response = generate_product_management_question([follow_up_question])
                print(f"Follow-up Response: {follow_up_response}")
                speak_text(f"Here is the follow-up response: {follow_up_response}")

            elif command == "answer":
                speak_text("Please provide your answer.")
                user_answer = listen_to_user()
                print(f"Your Answer: {user_answer}")

                # Confirm if user is done answering before evaluation
                speak_text("Do you want to add more to your answer before I evaluate it?")
                final_confirmation = listen_to_user()

                if "no" in final_confirmation or "done" in final_confirmation:
                    # Evaluate the user's response with GPT-4
                    evaluation = evaluate_answer_with_gpt4(generated_question, user_answer)
                    print(f"Evaluation Result:\n{evaluation}")
                    speak_text(f"Your response was evaluated. Here is the result. {evaluation}")
                    break
                else:
                    speak_text("Okay, please continue with your answer.")
                    additional_answer = listen_to_user()
                    user_answer += " " + additional_answer  # Append more to the answer
                    print(f"Updated Answer: {user_answer}")

            elif "stop" in user_input or "no" in user_input:
                break

            else:
                speak_text("Sorry, I did not understand. You can answer the question, ask for more information, or ask to rephrase the question.")
                print("Sorry, I did not understand. You can answer the question, ask for more information, or ask to rephrase the question.")

    elif choice == "2":
        speak_text("Please provide your project management interview question.")
        user_question = listen_to_user()
        print(f"Your Question: {user_question}")

        # Generate guidelines for answering the question with GPT-4
        guidelines = generate_guidelines_with_gpt4(user_question)
        print(f"Guidelines for Answering the Question:\n{guidelines}")
        speak_text(guidelines)

    else:
        print("Invalid option. Please restart the program and choose either 1 or 2.")
        speak_text("Invalid option. Please restart the program and choose either 1 or 2.")

if __name__ == "__main__":
    main()


Welcome to the Project Management Bot!
Please select an option:
1. Let the bot ask an interview question and evaluate your answer.
2. Provide your own project management interview question and get guidelines on how to answer it.
Enter 1 or 2: 2
Listening...
Your Question: can you tell me about a time when you had to like consider prioritising one product over another
Guidelines for Answering the Question:
**Good Answer**: A competent response might focus on a specific instance where the interviewee had to make a difficult decision between two products. They might say, "In my previous role as a Product Manager at XYZ Corp, we had two products in development at the same time - a mobile app and a web-based platform. Due to resource constraints, we had to prioritise one. I chose to prioritise the mobile app because our market research showed a higher demand for mobile solutions. The decision resulted in a successful product launch with positive user feedback."

**Better Answer**: A better 